# Classification of breast cancer data  [35 points]

You will implement a Support Vector Machine (SVM) model for the classification of breast cancer data and compare its performance with K-nearest Neighbors.

#### Loading the data

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
## loading the raw data using pandas
bc=...
bc.head()

#### Pre-process and understand data (5 pts)

The data label is the diagnosis (M = malignant, B = benign).

**Question (2pts):** Convert diagnosis column to binary 0-1 label where 1 incidates malignancy.

In [ ]:
## convert diagnosis to numerical label (M = malignant, B = benign)
y=...
y.head() # response variable

**Question (3pts):** Plot the first 2 features against diagnosis. Is the data linearly separable and what happens if we fit a hard-margin SVM to the data?

In [ ]:
##plotting the the first 2 features against diagnosis

#### Splitting the data: training and test 

In [ ]:
from sklearn.model_selection import train_test_split

X = ...
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

## 1. Support Vector Machine (SVM) (20 pts)

**Question (15pts):** Train and test a SVM model with the following requirements:
- Use GridSearchCV to find the best combination of model hyper-parameters: kernel: {linear, rbf}; for both kernel, C:{1, 10, 100, 1000}; for rbf kernel 'gamma' = {1e-3, 1e-4, 1e-5}.
- Try both accuracy and recall metric as scoring function in GridSearchCV
- Report the best hyper-parameter setting for each metric
- Report and plot the confusion matrix of the best estimator for each metric

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

svc=...
param_grid = ...
scores = ...
for sc in scores:
    grid=...
    print("# Tuning hyper-parameters for %s" % sc)
    grid.fit(X_train,y_train)
    
    svc = ...
    
    y_pred_svc =...
    ## computing and plotting confusion matrix

**Question (5pts):** Fit an SVM model to a slice of data that contains only the first 2 features and visualize decision boudary based on these two features.

In [ ]:
from sklearn import metrics

## for display purposes, we fit the model on the first two features
svc.fit(...)

## Plotting the decision boundary for all data (both train and test)
...

plt.title('SVC')
plt.show()

### 2. K-nearest Neighbors classifier (10pts)

**Question:** Repeat all the tasks for SVM to train a kNN classifier where the hyper-parameter is only k (number of neighbors) ranging from 1 to 50 in kNN. Compare the results with those of SVM, which classifier gives better results?

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn=...

k_range=...
param_grid=dict(n_neighbors=k_range)
scores = ...
for sc in scores:
    grid_knn=...
    
    grid_knn.fit(X_train,y_train)
    
    knn = ...
    
    y_pred_knn =...
    ## computing and plotting confusion matrix
    

In [ ]:
knn.fit(X_train[:,0:2], y_train)
## Plotting the decision boundary for all data (both train and test)


plt.title('KNN')
plt.show()